In [1]:
%matplotlib inline
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import problem
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv(
    os.path.join('data', 'train.csv.bz2')
)
data.loc[:, 'DateOfDeparture'] = pd.to_datetime(data.loc[:, 'DateOfDeparture'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8902 entries, 0 to 8901
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DateOfDeparture   8902 non-null   datetime64[ns]
 1   Departure         8902 non-null   object        
 2   Arrival           8902 non-null   object        
 3   WeeksToDeparture  8902 non-null   float64       
 4   log_PAX           8902 non-null   float64       
 5   std_wtd           8902 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 417.4+ KB


In [4]:
X, y = problem.get_train_data()
print(X.shape)
print(y.shape)
print(y[:10])

(8902, 5)
(8902,)
[12.33129622 10.77518151 11.08317675 11.16926784 11.26936373 12.07364919
 11.17393612  9.5999524   9.17564514 10.73431968]


In [5]:
from sklearn.preprocessing import FunctionTransformer

def _encode_dates(X):
    # With pandas < 1.0, we wil get a SettingWithCopyWarning
    # In our case, we will avoid this warning by triggering a copy
    # More information can be found at:
    # https://github.com/scikit-learn/scikit-learn/issues/16191
    X_encoded = X.copy()

    # Make sure that DateOfDeparture is of datetime format
    X_encoded.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
    # Encode the DateOfDeparture
    X_encoded.loc[:, 'year'] = X_encoded['DateOfDeparture'].dt.year
    X_encoded.loc[:, 'month'] = X_encoded['DateOfDeparture'].dt.month
    X_encoded.loc[:, 'day'] = X_encoded['DateOfDeparture'].dt.day
    X_encoded.loc[:, 'weekday'] = X_encoded['DateOfDeparture'].dt.weekday
    X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week
    X_encoded.loc[:, 'n_days'] = X_encoded['DateOfDeparture'].apply(
        lambda date: (date - pd.to_datetime("1970-01-01")).days
    )
    # Once we did the encoding, we will not need DateOfDeparture
#     return X_encoded.drop(columns=["DateOfDeparture"])
    return X_encoded

date_encoder = FunctionTransformer(_encode_dates)
X = date_encoder.fit_transform(X)

print(X.shape)
print(y.shape)

<ipython-input-5-a91cca9965dc>:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week


(8902, 11)
(8902,)


In [6]:
__file__ = os.path.join('submissions', 'starting_kit', 'estimator.py')
filepath = os.path.join(os.path.dirname(__file__), 'external_data.csv')
filepath

'submissions/starting_kit/external_data.csv'

In [7]:
class MergeTransformer():
    """Custom scaling transformer"""
    
    def read_csv_ramp(self, parse_dates=["Date"]):
        self.filepath = os.path.join(
            self.filepath, self.filename
        )
        
        data = pd.read_csv(os.path.join('data', 'train.csv.bz2'))
        if parse_dates is not None:
            ext_data = pd.read_csv(self.filepath, parse_dates=parse_dates)
        else:
            ext_data = pd.read_csv(self.filepath)
        return ext_data
    
    def merge_external_data(self):

        X = self.X.copy()  # to avoid raising SettingOnCopyWarning
        # Make sure that DateOfDeparture is of dtype datetime
#         X.loc[:, "DateOfDeparture"] = pd.to_datetime(X['DateOfDeparture'])

        if not(self.filename is None):
            self.X_ext = self.read_csv_ramp(parse_dates=self.parse_dates)

        if self.cols_to_keep != 'all':
            self.X_ext = self.X_ext[self.cols_to_keep]

        if self.cols_to_rename != None:
            self.X_ext = self.X_ext.rename(columns=self.cols_to_rename)

        X_merged = pd.merge(
            X, self.X_ext, how=self.how, on=self.on, sort=False
        )
        return X_merged

    
    def __init__(self, X_ext=None, filename=None, filepath='submissions/starting_kit/', cols_to_keep='all', cols_to_rename=None, how='left', on=None, parse_dates=None):
#         super().__init__(func)
        self.X_ext = X_ext
        self.filename = filename
        self.filepath = filepath
        self.cols_to_keep = cols_to_keep
        self.cols_to_rename = cols_to_rename
        self.how = how
        self.on = on
        self.parse_dates = parse_dates
        
    def fit_transform(self, X):
        self.fit(X)
        return self.transform()

    def fit(self, X):
        self.X = X

    def transform(self):
        return self.merge_external_data()

In [8]:
merge_transform = MergeTransformer(
    X_ext=None, 
    filename='external_data.csv',
    filepath='submissions/starting_kit/',
    cols_to_rename={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'}, 
    how='left',
    on=['DateOfDeparture', 'Arrival'],
    parse_dates=['Date'])

X = merge_transform.fit_transform(X)

print(X.shape)
print(y.shape)

(8902, 33)
(8902,)


In [9]:
coordinates_data = pd.read_csv('data/list-of-airports-in-united-states-of-america-hxl-tags-1.csv', index_col=0)
coordinates_data.loc[:, 'iso_region'] = coordinates_data.loc[:, 'iso_region'].str.strip('US-')
coordinates_data.drop('#meta +id', axis=0, inplace=True)

print(coordinates_data.shape)
coordinates_data.drop_duplicates(['iata_code'], inplace=True)
print('number unique airports', len(coordinates_data.name.unique()))
print(coordinates_data.shape)
# coordinates_data.head()
print('number unique lat', len(coordinates_data.latitude_deg.unique()))
print('number unique long', len(coordinates_data.longitude_deg.unique()))
print('number unique iata code', len(coordinates_data.iata_code.unique()))

(22216, 19)
number unique airports 1969
(1983, 19)
number unique lat 1980
number unique long 1981
number unique iata code 1983


In [10]:
merge_transform = MergeTransformer(
    X_ext=coordinates_data, 
    filename=None,
    filepath=None,
    cols_to_keep=['latitude_deg', 'longitude_deg', 'iata_code', 'iso_region'], 
    cols_to_rename={'iata_code': 'Departure',
                    'latitude_deg': 'latitude_departure',
                    'longitude_deg': 'longitude_departure',
                    'iso_region': 'state'},
    how='inner',
    on=['Departure'],
    parse_dates=None)

X = merge_transform.fit_transform(X)

print(X.shape)
print(y.shape)

X.drop_duplicates()
print('drop dup 1', X.shape)

merge_transform = MergeTransformer(
    X_ext=coordinates_data, 
    filename=None,
    filepath=None,
    cols_to_keep=['latitude_deg', 'longitude_deg', 'iata_code'], 
    cols_to_rename={'iata_code': 'Arrival', 
                    'latitude_deg': 'latitude_arrival', 
                    'longitude_deg': 'longitude_arrival'}, 
    how='inner',
    on=['Arrival'],
    parse_dates=None)

X = merge_transform.fit_transform(X)

import geopy.distance

X['distance'] = X.apply(lambda x: geopy.distance.geodesic(
    (x.latitude_departure, x.longitude_departure), 
    (x.latitude_arrival, x.longitude_arrival)).km, axis=1)
X

print(X.shape)

X.drop_duplicates()
print('drop dup 2', X.shape)


(8902, 36)
(8902,)
drop dup 1 (8902, 36)
(8902, 39)
drop dup 2 (8902, 39)


In [11]:
import holidays

us_holidays = holidays.US(years=2011, state='CA')
for key, value in us_holidays.items():
    print(f"key = {key}, value = {value}")

key = 2011-01-01, value = New Year's Day
key = 2010-12-31, value = New Year's Day (Observed)
key = 2011-01-17, value = Martin Luther King Jr. Day
key = 2011-02-21, value = Washington's Birthday
key = 2011-03-31, value = César Chávez Day
key = 2011-05-30, value = Memorial Day
key = 2011-07-04, value = Independence Day
key = 2011-09-05, value = Labor Day
key = 2011-10-10, value = Columbus Day
key = 2011-11-11, value = Veterans Day
key = 2011-11-24, value = Thanksgiving
key = 2011-12-25, value = Christmas Day
key = 2011-12-26, value = Christmas Day (Observed)


In [12]:
X.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,latitude_departure,longitude_departure,state,latitude_arrival,longitude_arrival,distance
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012,6,19,1,25,15510,34,29,24,22,21,19,82,63,44,1012,1010,1009,16,16,16,48,29,60.0,0.00,5,NaN,161,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797
1,2012-07-19,ORD,DFW,11.130435,7.027895,2012,7,19,3,29,15540,38,32,26,21,18,16,74,51,27,1018,1016,1013,16,16,16,24,15,34.0,0.00,1,NaN,208,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797
2,2011-12-09,ORD,DFW,12.652174,9.527991,2011,12,9,4,49,15317,9,5,1,6,4,-2,100,85,70,1031,1025,1021,16,10,1,27,11,32.0,0.00,7,Fog,47,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797
3,2012-03-26,ORD,DFW,11.263158,9.170254,2012,3,26,0,13,15425,28,21,14,15,13,12,80,62,43,1017,1015,1014,16,16,16,27,11,35.0,0.00,2,NaN,170,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797
4,2012-10-17,ORD,DFW,10.136364,7.716991,2012,10,17,2,42,15630,31,23,16,18,13,5,84,55,26,1008,1003,1000,16,16,16,42,26,53.0,0.00,4,NaN,235,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797


In [13]:
states = X.loc[:, 'state'].unique()
years = [2011, 2012, 2013]

X['bank_holidays'] = X.apply(lambda x: x.DateOfDeparture in holidays.US(years = x.year, state=x.state), axis=1)
print(X.shape)
print(y.shape)

(8902, 40)
(8902,)


In [14]:
school_holidays = pd.read_csv('data/holidays.csv', sep=';', parse_dates=['date'])

In [15]:
merge_transform = MergeTransformer(
    X_ext=school_holidays, 
    filename=None,
    filepath=None,
    cols_to_keep=['date', 'is_vacation'], 
    cols_to_rename={'date': 'DateOfDeparture', 'is_vacation': 'school_holidays'},
    how='left',
    on=['DateOfDeparture'],
    parse_dates=None)

X = merge_transform.fit_transform(X)
print(X.shape)
print(y.shape)

(8902, 41)
(8902,)


In [16]:
X.loc[:, 'holidays'] = X.loc[:, 'bank_holidays'] | X.loc[:, 'school_holidays']
X.drop(['bank_holidays', 'school_holidays'], inplace=True, axis=1)

In [17]:
X.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,latitude_departure,longitude_departure,state,latitude_arrival,longitude_arrival,distance,holidays
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012,6,19,1,25,15510,34,29,24,22,21,19,82,63,44,1012,1010,1009,16,16,16,48,29,60.0,0.00,5,NaN,161,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,True
1,2012-07-19,ORD,DFW,11.130435,7.027895,2012,7,19,3,29,15540,38,32,26,21,18,16,74,51,27,1018,1016,1013,16,16,16,24,15,34.0,0.00,1,NaN,208,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,True
2,2011-12-09,ORD,DFW,12.652174,9.527991,2011,12,9,4,49,15317,9,5,1,6,4,-2,100,85,70,1031,1025,1021,16,10,1,27,11,32.0,0.00,7,Fog,47,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False
3,2012-03-26,ORD,DFW,11.263158,9.170254,2012,3,26,0,13,15425,28,21,14,15,13,12,80,62,43,1017,1015,1014,16,16,16,27,11,35.0,0.00,2,NaN,170,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False
4,2012-10-17,ORD,DFW,10.136364,7.716991,2012,10,17,2,42,15630,31,23,16,18,13,5,84,55,26,1008,1003,1000,16,16,16,42,26,53.0,0.00,4,NaN,235,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False


In [18]:
airports_rank = pd.read_csv('data/airports_passengers.csv', sep=';', encoding = "utf-8")
airports_rank.head()

,Rank,Airports (large hubs),IATA,Major city served,State,2019,2018,2017,2016
0,1,Hartsfield-Jackson Atlanta International Airport,ATL,Atlanta,GA,NaN,51866464,50251964,50501858
1,2,Los Angeles International Airport,LAX,Los Angeles,CA,NaN,42626783,41232432,39636042
2,3,O'Hare International Airport,ORD,Chicago,IL,NaN,39874879,38593028,37589899
3,4,Dallas/Fort Worth International Airport,DFW,Dallas,TX,NaN,32800721,31816933,31283579
4,5,Denver International Airport,DEN,Denver,CO,NaN,31363573,29809097,28267394


In [19]:
merge_transform = MergeTransformer(
    X_ext=airports_rank, 
    filename=None,
    filepath=None,
    cols_to_keep=['2016', 'IATA'], 
    cols_to_rename={'IATA': 'Departure', '2016': 'airport_departure_capacity'},
    how='left',
    on=['Departure'],
    parse_dates=None)

X = merge_transform.fit_transform(X)
print(X.shape)
print(y.shape)

merge_transform = MergeTransformer(
    X_ext=airports_rank, 
    filename=None,
    filepath=None,
    cols_to_keep=['2016', 'IATA'], 
    cols_to_rename={'IATA': 'Arrival', '2016': 'airport_arrival_capacity'},
    how='left',
    on=['Arrival'],
    parse_dates=None)

X = merge_transform.fit_transform(X)
print(X.shape)
print(y.shape)

(8902, 41)
(8902,)
(8902, 42)
(8902,)


In [20]:
X

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,latitude_departure,longitude_departure,state,latitude_arrival,longitude_arrival,distance,holidays,airport_departure_capacity,airport_arrival_capacity
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012,6,19,1,25,15510,34,29,24,22,21,19,82,63,44,1012,1010,1009,16,16,16,48,29,60.0,0.00,5,NaN,161,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,True,37589899,31283579
1,2012-07-19,ORD,DFW,11.130435,7.027895,2012,7,19,3,29,15540,38,32,26,21,18,16,74,51,27,1018,1016,1013,16,16,16,24,15,34.0,0.00,1,NaN,208,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,True,37589899,31283579
2,2011-12-09,ORD,DFW,12.652174,9.527991,2011,12,9,4,49,15317,9,5,1,6,4,-2,100,85,70,1031,1025,1021,16,10,1,27,11,32.0,0.00,7,Fog,47,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579
3,2012-03-26,ORD,DFW,11.263158,9.170254,2012,3,26,0,13,15425,28,21,14,15,13,12,80,62,43,1017,1015,1014,16,16,16,27,11,35.0,0.00,2,NaN,170,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579
4,2012-10-17,ORD,DFW,10.136364,7.716991,2012,10,17,2,42,15630,31,23,16,18,13,5,84,55,26,1008,1003,1000,16,16,16,42,26,53.0,0.00,4,NaN,235,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8897,2011-12-10,LGA,MIA,13.576923,9.377305,2011,12,10,5,49,15318,28,26,23,22,19,18,93,76,58,1020,1019,1017,16,16,16,27,13,35.0,0.25,6,NaN,71,40.77719879,-73.87259674,NY,25.79319953918457,-80.29060363769531,1764.808838,False,14706123,20875813
8898,2012-08-25,LGA,MIA,14.666667,9.876156,2012,8,25,5,34,15577,29,26,23,25,23,19,94,82,70,1015,1013,1012,16,14,3,29,12,42.0,30.23,8,Rain,32,40.77719879,-73.87259674,NY,25.79319953918457,-80.29060363769531,1764.808838,True,14706123,20875813
8899,2013-01-30,LGA,MIA,14.214286,9.255057,2013,1,30,2,5,15735,27,24,22,20,18,17,90,74,58,1020,1018,1016,16,16,16,37,16,52.0,0.00,3,Rain,146,40.77719879,-73.87259674,NY,25.79319953918457,-80.29060363769531,1764.808838,False,14706123,20875813
8900,2011-11-06,LGA,MIA,12.458333,8.607273,2011,11,6,6,44,15284,28,24,20,19,18,17,90,72,54,1019,1017,1015,16,16,13,34,16,45.0,1.78,5,Rain,36,40.77719879,-73.87259674,NY,25.79319953918457,-80.29060363769531,1764.808838,False,14706123,20875813


In [21]:
unemployement = pd.read_csv("data/unemployment_rate.csv", sep=';', index_col='State')
unemployement.stack()
unemployement = pd.DataFrame(unemployement.stack()).reset_index(drop=False)
unemployement.rename(columns={"State": "State", "level_1": "Date", 0: "UnemployementRate"}, inplace=True)
unemployement

,State,Date,UnemployementRate
0,Alabama,01/01/2011,9.3
1,Alabama,01/02/2011,9.3
2,Alabama,01/03/2011,9.2
3,Alabama,01/04/2011,9.3
4,Alabama,01/05/2011,9.6
...,...,...,...
1867,Wyoming,01/08/2013,4.6
1868,Wyoming,01/09/2013,4.5
1869,Wyoming,01/10/2013,4.6
1870,Wyoming,01/11/2013,4.4


In [22]:
states_codes = pd.read_csv("data/states.csv")

merge_transform = MergeTransformer(
    X_ext=states_codes, 
    filename=None,
    filepath=None, 
    cols_to_rename={'State': 'State'},
    how='left',
    on=['State'],
    parse_dates=None)

unemployement = merge_transform.fit_transform(unemployement)
unemployement

,State,Date,UnemployementRate,Abbreviation
0,Alabama,01/01/2011,9.3,AL
1,Alabama,01/02/2011,9.3,AL
2,Alabama,01/03/2011,9.2,AL
3,Alabama,01/04/2011,9.3,AL
4,Alabama,01/05/2011,9.6,AL
...,...,...,...,...
1867,Wyoming,01/08/2013,4.6,WY
1868,Wyoming,01/09/2013,4.5,WY
1869,Wyoming,01/10/2013,4.6,WY
1870,Wyoming,01/11/2013,4.4,WY


In [23]:
unemployement['DateOfDeparture'] = pd.to_datetime(unemployement['Date'], format='%d/%m/%Y')
date_encoder = FunctionTransformer(_encode_dates)
unemployement = date_encoder.fit_transform(unemployement)
unemployement.drop(['State', 'Date', 'DateOfDeparture', 'day', 'weekday', 'week', 'n_days'], axis=1, inplace=True)
unemployement

<ipython-input-5-a91cca9965dc>:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week


,UnemployementRate,Abbreviation,year,month
0,9.3,AL,2011,1
1,9.3,AL,2011,2
2,9.2,AL,2011,3
3,9.3,AL,2011,4
4,9.6,AL,2011,5
...,...,...,...,...
1867,4.6,WY,2013,8
1868,4.5,WY,2013,9
1869,4.6,WY,2013,10
1870,4.4,WY,2013,11


In [24]:
X
print(X.shape)
print(y.shape)

(8902, 42)
(8902,)


In [25]:
merge_transform = MergeTransformer(
    X_ext=unemployement, 
    filename=None,
    filepath=None,
    cols_to_rename={'Abbreviation': 'state'},
    how='left',
    on=['year', 'month', 'state'],
    parse_dates=None)

X = merge_transform.fit_transform(X)
print(X.shape)
print(y.shape)

(8902, 43)
(8902,)


In [26]:
X.drop('Events', axis=1, inplace=True)

In [27]:
X.shape

(8902, 42)

In [28]:
sdate = '2011-09-01'
edate = '2013-03-05'
date_range = pd.date_range(sdate, edate, freq='d')
dates = pd.DataFrame(date_range, columns=['Dates'])

oil = pd.read_csv('data/oil_price.csv')
oil.loc[:, 'date'] = pd.to_datetime(oil.loc[:, 'date'], format='%d/%m/%Y')
oil.dropna(inplace=True)

merge_transform = MergeTransformer(
    X_ext=oil, 
    filename=None,
    filepath=None,
    cols_to_rename={'date': 'Dates'},
    how='left',
    on=['Dates'],
    parse_dates=None)

oil_dates = merge_transform.fit_transform(dates)
oil_dates.interpolate(method='linear', inplace=True)

oil_dates


,Dates,value
0,2011-09-01,88.930000
1,2011-09-02,86.450000
2,2011-09-03,86.342500
3,2011-09-04,86.235000
4,2011-09-05,86.127500
...,...,...
547,2013-03-01,90.680000
548,2013-03-02,90.493333
549,2013-03-03,90.306667
550,2013-03-04,90.120000


In [29]:
merge_transform = MergeTransformer(
    X_ext=oil_dates, 
    filename=None,
    filepath=None,
    cols_to_rename={'Dates': 'DateOfDeparture', 'value': 'OilPrice'},
    how='left',
    on=['DateOfDeparture'],
    parse_dates=None)

X = merge_transform.fit_transform(X)
X.loc[:, 'OilPrice'].isna().sum()
print(X.shape)
print(y.shape)

(8902, 43)
(8902,)


In [30]:
X.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,latitude_departure,longitude_departure,state,latitude_arrival,longitude_arrival,distance,holidays,airport_departure_capacity,airport_arrival_capacity,UnemployementRate,OilPrice
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012,6,19,1,25,15510,34,29,24,22,21,19,82,63,44,1012,1010,1009,16,16,16,48,29,60.0,0.00,5,161,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,True,37589899,31283579,8.7,84.222
1,2012-07-19,ORD,DFW,11.130435,7.027895,2012,7,19,3,29,15540,38,32,26,21,18,16,74,51,27,1018,1016,1013,16,16,16,24,15,34.0,0.00,1,208,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,True,37589899,31283579,8.9,92.846
2,2011-12-09,ORD,DFW,12.652174,9.527991,2011,12,9,4,49,15317,9,5,1,6,4,-2,100,85,70,1031,1025,1021,16,10,1,27,11,32.0,0.00,7,47,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579,9.8,99.410
3,2012-03-26,ORD,DFW,11.263158,9.170254,2012,3,26,0,13,15425,28,21,14,15,13,12,80,62,43,1017,1015,1014,16,16,16,27,11,35.0,0.00,2,170,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579,8.8,107.030
4,2012-10-17,ORD,DFW,10.136364,7.716991,2012,10,17,2,42,15630,31,23,16,18,13,5,84,55,26,1008,1003,1000,16,16,16,42,26,53.0,0.00,4,235,41.97859955,-87.90480042,IL,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579,8.8,92.214


In [31]:
X.drop('state', axis=1, inplace=True)
X.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,latitude_departure,longitude_departure,latitude_arrival,longitude_arrival,distance,holidays,airport_departure_capacity,airport_arrival_capacity,UnemployementRate,OilPrice
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012,6,19,1,25,15510,34,29,24,22,21,19,82,63,44,1012,1010,1009,16,16,16,48,29,60.0,0.00,5,161,41.97859955,-87.90480042,32.89680099487305,-97.03800201416016,1290.346797,True,37589899,31283579,8.7,84.222
1,2012-07-19,ORD,DFW,11.130435,7.027895,2012,7,19,3,29,15540,38,32,26,21,18,16,74,51,27,1018,1016,1013,16,16,16,24,15,34.0,0.00,1,208,41.97859955,-87.90480042,32.89680099487305,-97.03800201416016,1290.346797,True,37589899,31283579,8.9,92.846
2,2011-12-09,ORD,DFW,12.652174,9.527991,2011,12,9,4,49,15317,9,5,1,6,4,-2,100,85,70,1031,1025,1021,16,10,1,27,11,32.0,0.00,7,47,41.97859955,-87.90480042,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579,9.8,99.410
3,2012-03-26,ORD,DFW,11.263158,9.170254,2012,3,26,0,13,15425,28,21,14,15,13,12,80,62,43,1017,1015,1014,16,16,16,27,11,35.0,0.00,2,170,41.97859955,-87.90480042,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579,8.8,107.030
4,2012-10-17,ORD,DFW,10.136364,7.716991,2012,10,17,2,42,15630,31,23,16,18,13,5,84,55,26,1008,1003,1000,16,16,16,42,26,53.0,0.00,4,235,41.97859955,-87.90480042,32.89680099487305,-97.03800201416016,1290.346797,False,37589899,31283579,8.8,92.214


In [32]:
X.drop('Max Gust SpeedKm/h', inplace=True, axis=1)
X.columns

Index(['DateOfDeparture', 'Departure', 'Arrival', 'WeeksToDeparture',
       'std_wtd', 'year', 'month', 'day', 'weekday', 'week', 'n_days',
       'Max TemperatureC', 'Mean TemperatureC', 'Min TemperatureC',
       'Dew PointC', 'MeanDew PointC', 'Min DewpointC', 'Max Humidity',
       'Mean Humidity', 'Min Humidity', 'Max Sea Level PressurehPa',
       'Mean Sea Level PressurehPa', 'Min Sea Level PressurehPa',
       'Max VisibilityKm', 'Mean VisibilityKm', 'Min VisibilitykM',
       'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h', 'Precipitationmm',
       'CloudCover', 'WindDirDegrees', 'latitude_departure',
       'longitude_departure', 'latitude_arrival', 'longitude_arrival',
       'distance', 'holidays', 'airport_departure_capacity',
       'airport_arrival_capacity', 'UnemployementRate', 'OilPrice'],
      dtype='object')

In [33]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

pipe = make_pipeline(OneHotEncoder(),
                    StandardScaler(with_mean=False),
                    LinearRegression())

print(X.shape)
print(y.shape)

(8902, 41)
(8902,)


In [34]:
pipe.fit(X, y)
X.isna().sum()

DateOfDeparture               0
Departure                     0
Arrival                       0
WeeksToDeparture              0
std_wtd                       0
year                          0
month                         0
day                           0
weekday                       0
week                          0
n_days                        0
Max TemperatureC              0
Mean TemperatureC             0
Min TemperatureC              0
Dew PointC                    0
MeanDew PointC                0
Min DewpointC                 0
Max Humidity                  0
Mean Humidity                 0
Min Humidity                  0
Max Sea Level PressurehPa     0
Mean Sea Level PressurehPa    0
Min Sea Level PressurehPa     0
Max VisibilityKm              0
Mean VisibilityKm             0
Min VisibilitykM              0
Max Wind SpeedKm/h            0
Mean Wind SpeedKm/h           0
Precipitationmm               0
CloudCover                    0
WindDirDegrees                0
latitude

In [35]:
y_pred = pipe.predict(X)
print(y_pred[:10])
print(y[:10])
# pipe.score(X, y_pred)

[12.33129647 10.77518148 11.08317656 11.16926771 11.26936401 12.07364734
 11.17393611  9.59995246  9.17564511 10.73431953]
[12.33129622 10.77518151 11.08317675 11.16926784 11.26936373 12.07364919
 11.17393612  9.5999524   9.17564514 10.73431968]


In [38]:
X.to_csv('data/X_full_features.csv')

# Testing

In [40]:
X_test, y_test = problem.get_test_data()
print(X_test.shape, y_test.shape)
X_test.head()

(2226, 5) (2226,)


,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-05-21,LAS,ORD,12.000000,9.860938
1,2012-12-20,SFO,DEN,10.600000,9.954634
2,2012-11-01,LGA,DTW,11.950000,9.207977
3,2012-01-03,DEN,LAS,11.476190,9.352107
4,2012-11-19,LAX,ATL,13.444444,10.363892


# Happy ending

In [47]:
X_features = pd.read_csv('data/X_full_features.csv')
X_features.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_features.loc[:, 'DateOfDeparture'], format='%Y-%m-%d')

X_test.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_test.loc[:, 'DateOfDeparture'], format='%Y/%m/%d')
X_test.head()

merge_transform = MergeTransformer(
    X_ext=X_features, 
    filename=None,
    filepath=None,
    how='left',
    on=['DateOfDeparture', 'Departure', 'Arrival'],
    parse_dates=None)

X_test = merge_transform.fit_transform(X_test)

In [48]:
print(X_test.shape)
X_test.head()

(2226, 44)


,DateOfDeparture,Departure,Arrival,WeeksToDeparture_x,std_wtd_x,Unnamed: 0,WeeksToDeparture_y,std_wtd_y,year,month,day,weekday,week,n_days,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,latitude_departure,longitude_departure,latitude_arrival,longitude_arrival,distance,holidays,airport_departure_capacity,airport_arrival_capacity,UnemployementRate,OilPrice
0,2012-05-21,LAS,ORD,12.000000,9.860938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-12-20,SFO,DEN,10.600000,9.954634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-11-01,LGA,DTW,11.950000,9.207977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-01-03,DEN,LAS,11.476190,9.352107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-11-19,LAX,ATL,13.444444,10.363892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
